In [1]:
import numpy as np
import random
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
import pymongo
import json
from sklearn.metrics import mean_squared_error
from random import sample

In [ ]:
sample()

In [3]:
#從xlsx匯入資料
rec_data= pd.read_excel(r'./melt_data.xlsx',encoding='utf-16',index_col=0)

In [27]:
#sample train test data

train_idx=sample(rec_data.index.to_list(),int(len(rec_data)*0.7))
train_data=rec_data.loc[train_idx]
test_tmp=rec_data.index.isin(train_idx)
test_data=rec_data[~test_tmp]

In [26]:
train_data.head()

,user,variable,value
466263,39,聯邦銀行-紅利卡,0
30409,1270,中國信託-中信兄弟聯名卡,0
368425,214,凱基銀行-中壽聯名卡,0
254505,201,合作金庫-雙幣卡,0
253968,2313,合作金庫-漢來美食聯名卡,0


In [39]:
#melt data to df and fillna with 0
train_df=train_data.pivot_table(values='value',index='user',columns='variable').fillna(0)

In [51]:
train_df[1]

KeyError: 1

In [78]:
indices = pd.Series(train_df.columns)   #所有卡series
#card_index_list=indices[train_df.loc[1]==1]

print(train_df.columns[train_df.loc[0]==1].to_list())
print(train_df.columns[train_df.loc[1]==1].to_list())

['中國信託-LINE Pay卡', '台新銀行-FlyGo卡']
['中國信託-LINE Pay卡', '台新銀行-@GoGo iCash御璽卡', '花旗銀行-新世代花旗饗樂生活悠遊卡']


In [74]:
#item_base_rec
def CF_Item_Based_Evaluate(train,TopN):
    #imput data type:df
    #轉置df
    traindata=train
    traindata_T=traindata.T
    #計算各卡間的相似度
    card_sim=cosine_similarity(traindata_T,traindata_T)
    
    #找出已持有卡index
    rec_list={}
    for user in traindata.index:
           #所有卡series
        my_card=traindata.columns[traindata.loc[user]==1].to_list()#找持有卡
        indices = pd.Series(traindata_T.index)
        card_index_list=[indices[indices == name].index[0] for name in my_card]  #找出各卡index
        weighted_card_rec=np.zeros(len(traindata_T.index))   #創全0且為卡片數量長度的array
        for  i in card_index_list:
            weighted_card_rec +=card_sim[i]           #將持有卡於其他卡的相似度相加成推薦權重
        #排列相似度最高前TopN名卡片
        weighted_score=pd.Series(weighted_card_rec).sort_values(ascending = False)
        TopN_indexes=weighted_score.iloc[len(my_card):len(my_card)+TopN].index
        rec_card=[]
        for i in TopN_indexes:
            rec_card.append(indices[i])
        rec_list[user]=rec_card
    return rec_list

In [75]:
CF_Item_Based_Evaluate(train_df,10)

{0: ['台新銀行-@GoGo iCash御璽卡',
  '國泰世華-KOKO COMBO icash聯名卡',
  '玉山銀行-Pi 拍錢包信用卡',
  '花旗銀行-新世代花旗饗樂生活悠遊卡',
  '元大銀行-鑽金卡',
  '兆豐銀行-e秒刷鈦金卡',
  '永豐銀行-現金回饋卡',
  '台北富邦-J卡',
  '永豐銀行-DAWHO現金回饋信用卡',
  '安泰銀行-鈦金卡'],
 1: ['國泰世華-KOKO COMBO icash聯名卡',
  '元大銀行-鑽金卡',
  '台新銀行-FlyGo卡',
  '玉山銀行-Pi 拍錢包信用卡',
  '兆豐銀行-e秒刷鈦金卡',
  '滙豐銀行-現金回饋御璽卡',
  '安泰銀行-鈦金卡',
  '永豐銀行-現金回饋卡',
  '第一銀行-icash聯名卡',
  '安泰銀行-晶緻卡'],
 2: ['新光銀行-Lamigo信用卡',
  '上海商銀-小小兵Bello回饋卡',
  '渣打銀行-TheShoppingCard分期卡',
  '中國信託-寰遊美國運通卡',
  '國泰世華-尊榮御璽卡',
  '台新銀行-ETC聯名卡',
  '中國信託-酷玩卡',
  '聯邦銀行-法拉利無限卡',
  '台北富邦-富邦悍將悠遊聯名卡',
  '凱基銀行-無限卡'],
 3: ['中國信託-LINE Pay卡',
  '國泰世華-KOKO COMBO icash聯名卡',
  '元大銀行-鑽金卡',
  '台新銀行-FlyGo卡',
  '玉山銀行-Pi 拍錢包信用卡',
  '兆豐銀行-e秒刷鈦金卡',
  '第一銀行-icash聯名卡',
  '滙豐銀行-現金回饋御璽卡',
  '臺灣企銀-悠遊聯名卡',
  '安泰銀行-鈦金卡'],
 4: ['台新銀行-@GoGo iCash御璽卡',
  '國泰世華-KOKO COMBO icash聯名卡',
  '凱基銀行-無限卡',
  '花旗銀行-新世代花旗饗樂生活悠遊卡',
  '兆豐銀行-世界卡',
  '玉山銀行-玉山世界卡',
  '安泰銀行-鈦金卡',
  '安泰銀行-晶緻卡',
  '元大銀行-鑽金卡',
  '永豐銀行-現金回饋卡'],
 5: ['中國信託-LINE Pay卡',
  '國泰世華-KOKO COMBO icash聯名卡',
 

In [ ]:
#calculate TopN

In [7]:
for i in df.itertuples():
    print(i)

Pandas(Index=0, _1=0, _2=0, _3=0, _4=0, _5=0, _6=0, _7=1, _8=0, _9=0, _10=0, _11=0, _12=0, _13=0, _14=0, _15=0, _16=0, _17=0, _18=0, _19=0, _20=0, _21=0, _22=0, _23=0, _24=0, _25=0, _26=0, _27=0, _28=0, _29=0, _30=0, _31=0, _32=0, _33=0, _34=0, _35=0, _36=0, _37=0, _38=0, _39=0, _40=0, _41=0, _42=0, _43=0, _44=0, _45=0, _46=1, _47=0, _48=1, _49=0, _50=0, _51=0, _52=0, _53=0, _54=0, _55=0, _56=0, _57=0, _58=0, _59=0, _60=0, _61=0, _62=0, _63=0, _64=0, _65=0, _66=0, _67=0, _68=0, _69=0, _70=1, _71=0, _72=0, _73=0, _74=0, _75=0, _76=0, _77=0, _78=0, _79=0, _80=0, _81=0, _82=0, _83=0, _84=0, _85=0, _86=0, _87=0, _88=0, _89=0, _90=0, _91=0, _92=0, _93=0, _94=0, _95=0, _96=0, _97=0, 合庫JCB哆啦A夢=0, _99=0, _100=0, _101=0, _102=0, _103=0, _104=0, _105=0, _106=0, _107=0, _108=0, _109=0, _110=0, _111=0, _112=0, _113=0, _114=0, _115=0, _116=0, _117=0, _118=0, _119=0, _120=0, _121=0, _122=0, _123=0, _124=0, _125=0, _126=0, _127=0, _128=0, _129=0, _130=0, _131=0, _132=0, _133=0, _134=0, _135=0, _136=0

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [ ]:


reader = Reader(rating_scale=(0.5, 5))
data = Dataset.load_from_df(df[['userId', 'title', 'rating']], reader)
trainSet, testSet = train_test_split(data, test_size=.25, random_state=0)
algo = SVD(random_state=0)
algo.fit(trainSet)
predictions = algo.test(testSet)

def MAE(predictions):
        return accuracy.mae(predictions, verbose=False)
def RMSE(predictions):
        return accuracy.rmse(predictions, verbose=False)
    
print("RMSE: ", RMSE(predictions))
print("MAE: ", MAE(predictions))

In [3]:
SVD

NameError: name 'SVD' is not defined